In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [ ]:
from polars_ds.linear_models import LR

In [ ]:
model = LR()

In [ ]:
model.fit(x, y, null_policy="skip")

In [ ]:
import numpy as np

x = df.select("x1", "x2", "x3").to_numpy()


In [ ]:
np.any(np.isnan(x), axis = 1) | True

In [ ]:
from polars_ds.linear_models import LR

In [ ]:
model = LR(fit_bias = False, lambda_=0.1)
model

In [ ]:
model.fit_df(
    df, 
    features = ["x1", "x2", "x3"],
    target = "y",
    show_report = True
)

In [ ]:
model.predict_df(df)

In [ ]:
df.select(
    pds.query_lstsq(
        "x1", "x2", "x3",
        target = "y",
        method = "l2",
        solver = "svd",
        l2_reg = 0.1,
        add_bias = True
    )
)

In [ ]:
from polars_ds import transforms as t


df.select(

    t.polynomial_features(
        ["x1", "x2"],
        degree = 4, 
        interaction_only = False
    )

)

In [ ]:
kernel = [0.5] * 500

In [ ]:
%%timeit
df.select(
    pds.convolve("x1", kernel = kernel, method="direct").alias("convolve_1"),
)

In [ ]:
%%timeit
df.select(
    pds.convolve2("x1", kernel = kernel, method="direct").alias("convolve_1"),
)

In [ ]:
size = 1000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
    pds.random(0., 1.).alias("x2"),
    pds.random(0., 1.).alias("x3"),
).with_row_index().with_columns(
    y = pl.col("x1") + pl.col("x2") * 0.2 - 0.3 * pl.col("x3")
)

window_size = 5
min_valid_rows = 3

result = df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "exp(x2)", "x3",
        target = "y",
        window_size = window_size,
        min_valid_rows = min_valid_rows,
        null_policy = "skip" 
    ).alias("test")
).with_columns(
    pl.col("test").is_null().alias("is_null")
)

In [ ]:
df.select(
    "x1",
    pl.col("x1").shift(1).alias("x2")
).select(
    x1 = pl.col("x1") - pl.col("x2")
    , x2 = pl.col("x1").diff()
)

In [ ]:
df = pl.DataFrame({
    "a": [1,2,3,None]
})

df.select(
    pl.col("a").mean()
)